### Tutorial

[tutorial](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)

In [3]:
import gensim
import os
import collections
import smart_open
import random

# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.\
                TaggedDocument(gensim.utils.simple_preprocess(line), [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [4]:
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 4.25 s, sys: 281 ms, total: 4.53 s
Wall time: 2.32 s


In [5]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([ 0.07469384,  0.20438537,  0.05277047, -0.18572378,  0.13229951,
       -0.02090457,  0.01815359, -0.10901041, -0.03671757, -0.06418545,
       -0.20613658, -0.14565922, -0.02660019, -0.14839795, -0.06636003,
       -0.26222193,  0.22180766, -0.15623936, -0.06923521,  0.15165986,
       -0.06568333,  0.03122366,  0.02533489, -0.11533707,  0.18016511,
        0.10787751,  0.01791285,  0.07738844, -0.15631409,  0.00264642,
        0.01007171,  0.04655829,  0.15685908,  0.13112743,  0.09632727,
       -0.12145588,  0.01991077,  0.12894838,  0.06972929, -0.17885312,
        0.29637566,  0.07594626, -0.05257586, -0.18943392,  0.05332807,
        0.03900145,  0.13503976,  0.2273452 ,  0.11497773,  0.15150335],
      dtype=float32)

In [6]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [7]:
collections.Counter(ranks)

Counter({0: 292, 1: 8})

In [8]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [9]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index],\
                              ' '.join(train_corpus[sims[index][0]].words)))

Test Document (16): «beijing has abruptly withdrawn new car registration system after drivers demonstrated an unhealthy fixation with symbols of western military and industrial strength such as fbi and senior officials have been infuriated by popular demonstration of interest in american institutions such as the fbi particularly galling was one man choice of tmd which stands for theatre missile defence us designed missile system that is regularly vilified by chinese propaganda channels»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (66, 0.765294075012207): «argentina government has crumbled after at least people were killed and hundreds injured in nationwide riots argentina president fernando de la rua has resigned and called for national unity government with the opposition peronists the president resignation followed hours of rioting across the country people took to the streets protesting against the government economic austerity program argentina i